In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl == 0.12.0
!pip install stanza
!pip uninstall googletrans

!pip install langdetect
!pip install -U deep-translator
!pip install googletrans==4.0.0-rc1
!pip install google_trans_new


In [ ]:
!pip install datasets
!pip install trl==0.12.0
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoModelForSequenceClassification
import bitsandbytes as bnb
from deep_translator import GoogleTranslator
import time

In [ ]:
from huggingface_hub import login
login()

In [ ]:
path_annotations = f"dati.csv"
df = pd.read_csv(path_annotations, usecols=["Frase", "Name", "Start", "End", "Role", "Descriptor"])


df_protagonist = pd.DataFrame(columns=['statement','status'])
for index, row in df.iterrows():
  sub_classi=row['Descriptor'].split()
  entity = row['Name'].lower()
  sentence = row['Frase']
  full_input = f"Entity: {entity} [SEP] Sentence: {sentence}"

  if 'foreign' in sub_classi:
    sub_classi.remove('foreign')
    sub_classi.remove('adversary')
    sub_classi.append('Foreign Adversary')

  sub_classi = [f"{pr.capitalize()}" for pr in sub_classi]
  nuove_righe=[]

  for sb in sub_classi:
    new_row = {
        'statement': full_input,
        'status': sb
    }
    nuove_righe.append(new_row)

  temp_df = pd.DataFrame(nuove_righe)
  df_protagonist = pd.concat([df_protagonist, temp_df], ignore_index=True)

df=df_protagonist
df

In [ ]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=85).reset_index(drop=True).head(3000)

# Split the DataFrame
train_size =0.95
eval_size = 0.05

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

# Define the prompt generation functions
def generate_prompt(data_point):
    dati=data_point["statement"].split("[SEP]")
    return f""" You are a multi-label classifier.
            Given the following sentence:{dati[1].replace('Sentence:','')}
            Choose the labels that belong to the following entity: {dati[0].replace('Entity:','')}
            List of possible labels: 'Instigator','Conspirator','Tyrant','Foreign Adversary','Traitor','Spy','Saboteur','Corrupt','Incompetent','Terrorist','Deceiver','Bigot','Guardian','Martyr','Peacemaker','Rebel','Underdog','Virtuous','Forgotten','Exploited','Victim','Scapegoat'.
            Don't give me explanations, don't choose more than two labels
labels: {data_point["status"]}""".strip()

def generate_test_prompt(data_point):
    dati=data_point["statement"].split("[SEP]")
    return f""" You are a multi-label classifier.
            Given the following sentence:{dati[1].replace('Sentence:','')}
            Choose the labels that belong to the following entity: {dati[0].replace('Entity:','')}
            List of possible labels: 'Instigator','Conspirator','Tyrant','Foreign Adversary','Traitor','Spy','Saboteur','Corrupt','Incompetent','Terrorist','Deceiver','Bigot','Guardian','Martyr','Peacemaker','Rebel','Underdog','Virtuous','Forgotten','Exploited','Victim','Scapegoat'.
            Don't give me explanations, don't choose more than two labels
labels: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'status']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

X_train.status.value_counts()

In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])
train_data['text'][3]

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Configurazione di BitsAndBytes per la quantizzazione 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

# Caricamento del modello con quantizzazione 4-bit
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.problem_type = "multi_label_classification"
model.config.num_labels = 22

# Configurazioni del modello
model.config.use_cache = False
model.config.pretraining_tp = 1

# Caricamento del tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Imposta il token di padding come il token di fine sequenza
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

In [ ]:
output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=6,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine"
    report_to="wandb",
    eval_strategy="steps",
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

In [ ]:
trainer.train()


In [ ]:
def predictSingola(testo, model, tokenizer):
    y_pred = []
    categories = ['Instigator','Conspirator','Tyrant','Foreign Adversary','Traitor','Spy','Saboteur','Corrupt','Incompetent','Terrorist','Deceiver','Bigot','Guardian','Martyr','Peacemaker','Rebel','Underdog','Virtuous','Forgotten','Exploited','Victim','Scapegoat']

    prompt = generate_test_prompt(testo)
    pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=3,
                        temperature=0.1)

    result = pipe(prompt,return_full_text=False)
    result[0]['generated_text']=result[0]['generated_text'].replace(prompt,'')
    answer = result[0]['generated_text'].split("label:")[-1].strip()
    labels=result[0]['generated_text'].split("label:")
    labels_predette=' '.join(labels)
    # Determine the predicted category
    #DA RIVEDERE
    for category in categories:
        #answer.lower()
        if category.lower() in labels_predette.lower():
          y_pred.append(category)
    else:
          y_pred.append("none")

    return y_pred



def generate_test_prompt(testo):
    dati=testo.split("[SEP]")
    return f""" You are a multi-label classifier.
            Given the following sentence:{dati[1].replace('Sentence:','')}
            Choose the labels that belong to the following entity: {dati[0].replace('Entity:','')}
            List of possible labels: 'Instigator','Conspirator','Tyrant','Foreign Adversary','Traitor','Spy','Saboteur','Corrupt','Incompetent','Terrorist','Deceiver','Bigot','Guardian','Martyr','Peacemaker','Rebel','Underdog','Virtuous','Forgotten','Exploited','Victim','Scapegoat'.
            Don't give me explanations, don't choose more than two labels
            labels: """.strip()

In [ ]:
def tokenize_by_punctuation(text):
    tokens = re.split(r'([.!?]+)', text)

    phrases = []
    for i in range(0, len(tokens)-1, 2):
        sentence = tokens[i].strip() + tokens[i+1]
        phrases.append(sentence.strip())
    return phrases

def tokenize_text(text):
    tokens = text.splitlines()
    tokenized_list = []

    for token in tokens:
        sub_tokens = token.split('|')
        for sub_token in sub_tokens:
            cleaned_token = sub_token.strip()
            if cleaned_token:
                tokenized_list.append(cleaned_token)
    final_tokenized_list = []
    for frase in tokenized_list:
        sub_phrases = re.split(r'(?<=[.!?।])\s+', frase)
        final_tokenized_list.extend([s.strip() for s in sub_phrases if s.strip()])

    return final_tokenized_list

def trova_frase_contenente_stringa2(testo, start, end,isHI):
  if isHI:
      frasi=tokenize_text(testo)
      return trova_frase_con_parola(testo, frasi, start, end)
  else:
      frasi=tokenize_by_punctuation(testo)
      return trova_frase_con_parola(testo, frasi, start, end)


def trova_frase_con_parola(testo, frasi, inizio_parola, fine_parola):
    for frase in frasi:
      inizio_frase = testo.find(frase)
      fine_frase = inizio_frase + len(frase)
      if inizio_parola >= inizio_frase and fine_parola <= fine_frase:
          return (frase)
    return ''

  return sub_predette.issubset(sub_corrette) or PredizioneCorretta(descriptor,predizione)

def GetFraseDaAnalizzareConContesto(start, end, article, usa_stanza, lang):
    """
    Estrai la frase contenente un segmento di testo dato (start, end) insieme alle due frasi vicine.
    """
    if usa_stanza:
        sentences = get_sentences(article, lang)
    else:
        sentences = split_into_sentences(article)

    current_position = 0
    for i, sentence in enumerate(sentences):
        sentence_length = len(sentence)
        if current_position <= start < current_position + sentence_length:
            previous_sentence = sentences[i - 1].strip() if i > 0 else ""
            next_sentence = sentences[i + 1].strip() if i < len(sentences) - 1 else ""
            target_sentence = sentence.strip()

            combined_text = f"{previous_sentence} {target_sentence} {next_sentence}".strip()
            return combined_text
        current_position += sentence_length + 1

    # Nessuna frase trovata
    print("Nessuna frase trovata per offset:", start, end)
    return ""

In [ ]:
!pip install stanza
import pandas as pd
import stanza

# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences


def get_sentences(text, lang):
    """
    Crea una pipeline Stanza per una lingua specifica, segmenta il testo in frasi e restituisce la lista di frasi.

    Args:
        text (str): Il testo da segmentare.
        lang (str): Codice ISO-639-1 della lingua (es. 'en', 'ru', 'fr', ecc.).

    Returns:
        list: Lista di frasi segmentate.
    """
    stanza.download(lang)

    nlp = stanza.Pipeline(lang, processors='tokenize', use_gpu=True, verbose=False)

    doc = nlp(text)

    sentences = [sentence.text for sentence in doc.sentences]

    return sentences


def GetLingua(titolo):
  lingua='en'
  titolo=titolo.lower()
  if 'bg' in titolo:
    lingua='bg'
  elif 'en' in titolo:
    lingua='en'
  elif 'hi' in titolo:
    lingua='hi'
  elif 'pt' in titolo:
    lingua='pt'
  else:
    lingua='ru'

  return lingua


def Traduci(testo, lingua):
  return GoogleTranslator(source='pt', target='en').translate(testo)

In [ ]:
#FASE DI VALIDAZIONE
#INPUT: file con le annotazioni, cartello 'articoli' con l'insieme di articoli da analizzare
import re
# Leggi le annotazioni
rows_test=[]
path_annotations = f"subtask-1-entity-mentions.txt"

with open(path_annotations, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')

        if len(parts) > 6:
            parts = parts[:5] + [' '.join(parts[5:])]

        rows_test.append(parts)

df_righeEval = pd.DataFrame(rows_test, columns=["File", "Name", "Start", "End"])

df_evaluation = pd.DataFrame(columns=['input','Instigator','Conspirator','Tyrant','Foreign Adversary','Traitor','Spy','Saboteur','Corrupt','Incompetent','Terrorist','Deceiver','Bigot','Forgotten','Exploited','Victim','Scapegoat'
,'Guardian'
,'Martyr'
,'Peacemaker'
,'Rebel'
,'Underdog'
,'Virtuous'
])

for index, row in df_righeEval.iterrows():
        path = row["File"]
        nome = row["Name"].strip()
        start = row["Start"]
        end = row["End"]

        try:
            with open(f'articoli/{path}', "r", encoding="utf-8") as file:
                articolo = file.read()
        except Exception as e:
            print(f"Errore nell'apertura del file {path}: {e}")
            continue

        lingua="PT" #lingua di origine degli articoli
        frase=GetFraseDaAnalizzareConContesto(int(start),int(end),articolo,True,lingua)

        nome_tradotto=Traduci(nome,'pt')
        frase_tradotta=Traduci(frase,'pt')
        full_input = f"Entity: {nome_tradotto} [SEP] Sentence: {frase_tradotta}"

        sub=(predictSingola(full_input, model, tokenizer))

        if sub[0]=='none':
          sub=[]

        if 'none' in sub:
          sub.remove('none')

        #prendiamo una sola etichetta
        if len(sub)>1:
          sub = [sub[-1]]

        #ricerca del ruolo principale
        prima_sub=sub[0] if len(sub)>0 else ''
        classe='Antagonist'
        if prima_sub in ['Forgotten','Exploited','Victim','Scapegoat']:
          classe='Innocent'
        elif prima_sub in ['Guardian','Martyr','Peacemaker','Rebel','Underdog','Virtuous']:
          classe='Protagonist'

        with open('output', "a", encoding="utf-8") as out_file:
            line = f"{path}\t{row['Name']}\t{start}\t{end}\t{classe}\t" + '\t'.join(sub) + "\n"
            out_file.write(line)